In [2]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sys 
import os


In [19]:
sys.path.append(os.path.abspath('..')) 
base_dir = os.path.abspath(os.path.join(os.getcwd(), '../../data/clusters/'))

cluster_1 = pd.read_csv(base_dir + '/cluster_1.0.csv')
cluster_2 = pd.read_csv(base_dir + '/cluster_2.0.csv')
cluster_3 = pd.read_csv(base_dir + '/cluster_3.0.csv')
cluster_4 = pd.read_csv(base_dir + '/cluster_4.0.csv')
cluster_5 = pd.read_csv(base_dir + '/cluster_5.0.csv')
cluster_6 = pd.read_csv(base_dir + '/cluster_6.0.csv')
cluster_7 = pd.read_csv(base_dir + '/cluster_7.0.csv')
cluster_8 = pd.read_csv(base_dir + '/cluster_0.0.csv')

basket_path = os.path.abspath(os.path.join(os.getcwd(), '../../data/raw/'))


basket = pd.read_csv(basket_path + '/customer_basket.csv')

In [14]:
cluster_1.head()

,customer_id,kids_home,teens_home,number_complaints,distinct_stores_visited,spend_groceries,spend_electronics,typical_hour,spend_vegetables,spend_nonalcohol_drinks,...,spend_hygiene,spend_videogames,spend_petfood,total_distinct_products,percentage_of_products_bought_promotion,age,loyalty_member,years_as_customer,educ_years,gender_binary
0,43460,1.0,0.0,1.0,2.0,5015.0,1334.0,8.0,344.0,374.0,...,2115.0,1028.0,1987.0,2566.0,0.206227,36,0,10.0,12,0
1,23937,0.0,1.0,1.0,3.0,3097.0,4073.0,10.0,301.0,852.0,...,1306.0,1504.0,2018.0,1282.0,0.136168,55,1,15.0,12,0
2,15019,1.0,0.0,0.0,3.0,5164.0,448.0,7.0,411.0,598.0,...,2410.0,817.0,2170.0,903.0,0.383456,61,1,19.0,12,0
3,42461,1.0,1.0,1.0,2.0,1996.0,2138.0,9.0,295.0,950.0,...,2279.0,1237.0,2052.0,1014.0,0.250435,76,0,16.0,12,0
4,18510,0.0,0.0,0.0,2.0,3338.0,580.0,10.0,268.0,746.0,...,2105.0,1218.0,2317.0,1051.0,0.139292,54,1,19.0,12,0


In [10]:
for i in range(1, 9):
    print(eval(f"cluster_{i}").size)


113735
24196
163001
52670
121371
163093
249090
97474


In [24]:
def association_rules_pipeline(customers, basket, join_column='customer_id', list_column='list_of_goods',
                              min_support=0.2, metric='lift', min_threshold=1):
    """
    Perform the association rules pipeline on customer-basket data.

    Args:
        customers (pandas.DataFrame): The customer data.
        basket (pandas.DataFrame): The basket data.
        join_column (str, optional): The column name used for joining the customer and basket data. Defaults to 'customer_id'.
        list_column (str, optional): The column name containing the list of goods in the basket. Defaults to 'list_of_goods'.
        min_support (float, optional): The minimum support threshold for generating frequent itemsets. Defaults to 0.2.
        metric (str, optional): The metric used for evaluating association rules. Defaults to 'lift'.
        min_threshold (float, optional): The minimum threshold for the metric to consider a rule. Defaults to 1.

    Returns:
        pandas.DataFrame: The generated association rules.
    """
    # Merge customer and basket data on the specified join column
    data_merged = pd.merge(basket, customers, on=join_column, how='inner')

    # Extract transactions from the merged data
    transactions = data_merged[list_column].apply(lambda x: [item.strip() for item in x[1:-1].split(',')])

    # Convert transactions to transaction matrix using TransactionEncoder
    te = TransactionEncoder()
    te_fit = te.fit(transactions).transform(transactions)
    transactions_items = pd.DataFrame(te_fit, columns=te.columns_)

    # Generate frequent itemsets using Apriori algorithm
    frequent_itemsets = apriori(transactions_items, min_support=min_support, use_colnames=True)

    # Generate association rules from frequent itemsets
    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold)

    return rules

In [22]:
clusters = [cluster_1, cluster_2, cluster_3, cluster_4, cluster_5, cluster_6, cluster_7, cluster_8]  # replace with your actual cluster DataFrames


In [25]:
for i, cluster in enumerate(clusters):
    # Get the customer_ids for this cluster
    customer_ids = cluster['customer_id']

    # Filter the basket DataFrame to only include baskets for these customers
    cluster_baskets = basket[basket['customer_id'].isin(customer_ids)]

    # Run the association rules pipeline on these baskets
    rules = association_rules_pipeline(cluster_baskets, basket, join_column='customer_id', list_column='list_of_goods',
                                       min_support=0.2, metric='lift', min_threshold=1)

    # Save or print the rules as needed
    print(f"Association rules for cluster {i}:")
    print(rules)

KeyError: 'list_of_goods'